# Imports

In [9]:
import pandas as pd
import json

# GLM Table

In [10]:
# read csv
all_significant_contrasts = pd.read_csv('processed_data/glm/all_significant_contrasts.csv')

# remove the P>|z| and Significant column
all_significant_contrasts = all_significant_contrasts.drop(columns=['P>|z|', 'Significant', 'Chroma', '[0.025', '0.975]', 'mode'])

# rename the columns and capitalize the first letter of each word, except for 'z' and 'p_value_fdr'
all_significant_contrasts.columns = [col.replace('_', ' ').title() if col not in ['z', 'p_value_fdr'] else col for col in all_significant_contrasts.columns]

# rename the p_value_fdr column to $p_{fdr}$
all_significant_contrasts.rename(columns={'p_value_fdr': '$p_{fdr}$'}, inplace=True)

# rename Contrast Pair to 'Contrast'
all_significant_contrasts.rename(columns={'Contrast Pair': 'Contrast'}, inplace=True)

# rename 'z' to '$z$'
all_significant_contrasts.rename(columns={'z': '$z$'}, inplace=True)

# reorder the columns
all_significant_contrasts = all_significant_contrasts[['Contrast', 'Region', 'Ch Name', 'Coef.', 'Std.Err.', '$z$', '$p_{fdr}$']]

caption = "Table of contrast results from the GLM analysis. "

# Replace all substrings of 'Virt' with 'Virtual'
all_significant_contrasts['Contrast'] = all_significant_contrasts['Contrast'].str.replace('Virt', 'Virtual', regex=False)

# save as a latex table
with open('processed_data/glm/all_significant_contrasts.tex', 'w') as f:
    f.write(all_significant_contrasts.to_latex(index=False, float_format="%.3f", escape=False, column_format='|c|c|c|c|c|c|c|', longtable=True, caption=caption))

# remove every underscore from the latex table
with open('processed_data/glm/all_significant_contrasts.tex', 'r') as f:
    lines = f.readlines()
with open('processed_data/glm/all_significant_contrasts.tex', 'w') as f:
    for line in lines:
        if "fdr" in line:
            f.write(line.replace('>', '$>$'))
        else:
            f.write(line.replace('_', ' ').replace('>', '$>$'))

# Functional Connectivity Table

In [11]:
with open(f'processed_data\\spectral_connectivity_time\\group_level_t_tests_roi_contrasts.json', 'r') as f:
    contrast_json = json.load(f)

# create a dataframe to store the differences in the ratio of positive to negative t-values for each emotion contrast
contrast_df = pd.DataFrame(columns=['Contrast', 'Ratio'])

for mode in contrast_json.keys():
    if mode == 'all':
        continue
    for contrast_name, contrast_data in contrast_json[mode].items():
        num_positive_t = 0
        num_negative_t = 0
        for roi, values in contrast_data.items():
            # count the number of t-values that are positive and negative
            num_positive_t += sum(t > 0 for t in values['t_values'])
            num_negative_t += sum(t < 0 for t in values['t_values'])

        # Calculate the ratio of positive to negative t-values
        if (num_positive_t + num_negative_t) != 0:
            ratio = (num_positive_t - num_negative_t) / (num_positive_t + num_negative_t)
        else:
            ratio = 0  # avoid division by zero

        # Append the contrast name and ratio to the dataframe
        contrast_df = pd.concat(
            [contrast_df, pd.DataFrame({'Contrast': [contrast_name], 'Ratio': [ratio]})],
            ignore_index=True
        )

# Each contrast string is in one of the following formats:
# 'Condition1_Condition2' or 'Condition1_Condition2_Condition3_Condition4'
# In the first case, we want to replace the underscore with ' > ' and in the second case, we want to replace the first and last underscores with ' ' and the middle underscores with ' > '.
def format_contrast_string(contrast):
    parts = contrast.split('_')
    if len(parts) == 2:
        return f"{parts[0]} > {parts[1]}"
    elif len(parts) == 4:
        return f"{parts[0]} {parts[1]} > {parts[2]} {parts[3]}"
    else:
        return contrast

contrast_df['Contrast'] = contrast_df['Contrast'].apply(format_contrast_string)
contrast_df['Contrast'] = contrast_df['Contrast'].str.replace('Virt', 'Virtual', regex=False)

caption = "Ratio of positive to negative $t$-values for each contrast. The ratio is calculated as Ratio = (Number of Positive $t$-values - Number of Negative $t$-values) / (Number of Positive $t$-values + Number of Negative $t$-values). For condition1 > condition2, a positive ratio indicates that condition1 has more positive $t$-values than condition2, while a negative ratio indicates the opposite. "

# Save as a LaTeX table
with open('processed_data/spectral_connectivity_time/group_level_t_tests_roi_contrast_ratios.tex', 'w') as f:
    f.write(contrast_df.to_latex(index=False, float_format="%.3f", escape=False, column_format='|l|c|', longtable=True, caption=caption))

# Post-process LaTeX to escape underscores and replace '>' with LaTeX symbol
with open('processed_data/spectral_connectivity_time/group_level_t_tests_roi_contrast_ratios.tex', 'r') as f:
    lines = f.readlines()
with open('processed_data/spectral_connectivity_time/group_level_t_tests_roi_contrast_ratios.tex', 'w') as f:
    for line in lines:
        f.write(line.replace('_', '\\_').replace('>', '$>$'))

C:\Users\super\AppData\Local\Temp\ipykernel_25128\1563098282.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  contrast_df = pd.concat(


# ANOVA Table

In [12]:
# read csv
anova_table = pd.read_csv('processed_data/behavioural_responses/anova_table.csv')

caption = 'Two-way ANOVA results for the effect of Face Type and Emotion and their interaction on the correct responses. '

# set nan's to empty strings for better LaTeX formatting
anova_table = anova_table.fillna('')

# rename 'F' to '$F$' and 'PR(>F)' to 'PR(>$F$)'
anova_table.rename(columns={'F': '$F$', 'PR(>F)': 'PR(>$F$)'}, inplace=True)

# rename the first column to ''
anova_table.rename(columns={anova_table.columns[0]: ''}, inplace=True)

# save the anova table to a latex file
with open('processed_data/behavioural_responses/anova_table.tex', 'w') as f:
    f.write(anova_table.to_latex(index=False, float_format="%.5f", escape=False, column_format='|l|c|c|c|c|', longtable=True, caption=caption))

# remove every underscore from the latex table
with open('processed_data/behavioural_responses/anova_table.tex', 'r') as f:
    anova_table_latex = f.read().replace('_', '\\_').replace('>', '\\textgreater{}')
with open('processed_data/behavioural_responses/anova_table.tex', 'w') as f:
    f.write(anova_table_latex)